IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd

READ DATA

In [2]:
data  = pd.read_csv("tweets.csv")
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,568448023845572608,neutral,0.6423,NaN,NaN,Delta,NaN,smsuconn,NaN,0,@JetBlue yes he and all the princesses. Trying...,NaN,2015-02-19 08:32:24 -0800,"Alexandria, VA",Eastern Time (US & Canada)
1,567840103122509825,negative,1.0000,Cancelled Flight,1.0,US Airways,NaN,unpredictaylor,NaN,0,@USAirways UR service is so shitty. Pilot neve...,NaN,2015-02-17 16:16:45 -0800,NaN,Atlantic Time (Canada)
2,569710305707601920,negative,1.0000,Late Flight,1.0,US Airways,NaN,EAFiedler,NaN,0,@USAirways flight #703 Philly -&gt; Mia? Bwaha...,NaN,2015-02-22 20:08:16 -0800,Philly,Eastern Time (US & Canada)
3,570028083606122497,positive,1.0000,NaN,NaN,American,NaN,LSlay19,NaN,0,"@AmericanAir thanks, me too",NaN,2015-02-23 17:11:00 -0800,"NC by day, DC by night....",Quito
4,569358701317869571,negative,1.0000,Late Flight,1.0,Southwest,NaN,masonsontwiter2,NaN,0,@SouthwestAir yall have me sleeping in the air...,NaN,2015-02-21 20:51:07 -0800,NaN,Alaska


EXTRACTING LABELS AND TWEETS

In [3]:
reviews = np.array(data['text'])
labels = np.array(data['airline_sentiment'])
reviews

array(['@JetBlue yes he and all the princesses. Trying to get to the #glassslipperchallenge at WDW.',
       "@USAirways UR service is so shitty. Pilot never showed up so we waited hours because another pilot was supposed to come but didn't #Cancelled Flighted",
       '@USAirways flight #703 Philly -&gt; Mia? Bwahahaha! Delayed 5+ hrs, were in the air 30 mins then turned back around. #usairwaysfail',
       ...,
       "@JetBlue Even though this flight #226 didn't have much needed hot beverages 4 us NY-ers, the landing was super smooth 👍👍 😊 #happytweet",
       '@SouthwestAir Thanks. I did go through these motions shortly after my flight yesterday. I wonder how quickly flight attendants are notified.',
       '@United super lame that you charge $8 for tv on an international flight to the US. It isn’t 1997!'],
      dtype=object)

In [4]:
labels

array(['neutral', 'negative', 'negative', ..., 'positive', 'positive',
       'negative'], dtype=object)

CLEARING PUNCTUATIONS , HYPERLINKS , AND NUMBERS

In [5]:
from string import punctuation
def clean(words):
  #  punctuation = '@!"#$%&'\()*+,-./:;<=>?[\\]^_`{|}~'
    nword = []
    for word in words:
        if word[0] not in punctuation and word.find('http')==-1 and any(char.isdigit() for char in word)==False:
            #word = word.replace(punctuation, ' ')
            for pun in punctuation:
                word = word.replace(pun,'')
                word = word.lower()
                word = word.replace(" s "," ")
                
            nword.append(word)
    
    return nword

IMPORT SPACY's ENGLISH 

In [6]:
import spacy
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

FUNCTION FOR LEMMATIZATION

In [7]:
def lemma(sentence):
    doc = nlp(sentence)
   # print(doc)
    sent = " ".join([token.lemma_ for token in doc if token.is_stop==False])
   # print(sent)
# Extract the lemma for each token and join
    return sent

LEMMATIZING AND CLEANING 

In [8]:

revs = []
for re in reviews:
    #print(re)
    re = lemma(re)
    re = re.split()
    rn = clean(re)
    revs.append(rn)
    #print(rn)
revs

[['yes', 'princess', 'try', 'glassslipperchallenge', 'wdw'],
 ['ur',
  'service',
  'shitty',
  'pilot',
  'show',
  'wait',
  'hour',
  'pilot',
  'suppose',
  'come',
  'cancel',
  'flighted'],
 ['flight',
  'philly',
  'mia',
  'bwahahaha',
  'delay',
  'hrs',
  'air',
  'min',
  'turn',
  'usairwaysfail'],
 ['thanks'],
 ['sleep', 'airport', 'pm', 'tomorrow', 'thank'],
 ['fa', 'ground', 'personnel', 'great', 'job', 'weather', 'blame', 'talk'],
 ['raleigh', 'update', 'southern', 'states', 'advisory'],
 ['bad', 'company', 'fly', 'repeat', 'fly', 'credit', 'delay'],
 ['go',
  'miss',
  'hugely',
  'important',
  'meeting',
  'constant',
  'delay',
  'itscostingmeincome'],
 ['😃', 'cool'],
 ['stay', 'travel', 'ans', 'sleep', 'airport', 'floor'],
 ['expect', 'flight', 'dfw', 'cancel', 'flightled', 'tomorrow', 'afternoon'],
 ['ok', 'hope', 'thy', 'bahamas'],
 ['thank', 'get', 'find', 'airline', 'future'],
 [],
 ['thx', 'follow', 'send', 'dm'],
 ['look', 'well', 'service'],
 ['flight', 'ewr

making an array of all words

In [9]:
all_words = []
for re in revs:
    for word in re:
        all_words.append(word)
        
print(all_words)

['yes', 'princess', 'try', 'glassslipperchallenge', 'wdw', 'ur', 'service', 'shitty', 'pilot', 'show', 'wait', 'hour', 'pilot', 'suppose', 'come', 'cancel', 'flighted', 'flight', 'philly', 'mia', 'bwahahaha', 'delay', 'hrs', 'air', 'min', 'turn', 'usairwaysfail', 'thanks', 'sleep', 'airport', 'pm', 'tomorrow', 'thank', 'fa', 'ground', 'personnel', 'great', 'job', 'weather', 'blame', 'talk', 'raleigh', 'update', 'southern', 'states', 'advisory', 'bad', 'company', 'fly', 'repeat', 'fly', 'credit', 'delay', 'go', 'miss', 'hugely', 'important', 'meeting', 'constant', 'delay', 'itscostingmeincome', '😃', 'cool', 'stay', 'travel', 'ans', 'sleep', 'airport', 'floor', 'expect', 'flight', 'dfw', 'cancel', 'flightled', 'tomorrow', 'afternoon', 'ok', 'hope', 'thy', 'bahamas', 'thank', 'get', 'find', 'airline', 'future', 'thx', 'follow', 'send', 'dm', 'look', 'well', 'service', 'flight', 'ewr', 'stop', 'dallas', 'la', 'need', 'la', 'tonight', 'lmfao', 'flight', 'cheap', 'catch', 'happen', 'locate',

**tokenizing unique words**


In [10]:
from collections import Counter
counts = Counter(all_words)

sorted_counts = sorted(counts,key=counts.get,reverse=True)
word_to_int = {word:ii for ii,word in enumerate(sorted_counts,1)}

sz = len(word_to_int)

coded_revs = []
for review in revs:
    coded_revs.append([word_to_int[word] for word in review])

coded_revs


[[73, 3299, 18, 4547, 4548],
 [273, 4, 1334, 140, 278, 13, 3, 140, 193, 82, 5, 80],
 [1, 358, 553, 4549, 6, 103, 132, 53, 359, 680],
 [489],
 [326, 41, 218, 52, 2],
 [964, 220, 1335, 46, 227, 51, 641, 144],
 [1184, 100, 3300, 3301, 662],
 [29, 228, 11, 1336, 11, 142, 6],
 [24, 38, 2680, 847, 642, 1798, 6, 4550],
 [965, 312],
 [285, 49, 4551, 326, 41, 1008],
 [248, 1, 110, 5, 37, 52, 602],
 [194, 90, 3302, 2008],
 [2, 12, 70, 22, 465],
 [],
 [195, 91, 50, 67],
 [58, 155, 4],
 [1, 210, 178, 229, 412, 16, 412, 111],
 [2296],
 [1, 603, 526],
 [106, 733],
 [17, 23, 1185, 136, 879, 375, 129, 207, 25, 680],
 [82, 266, 4, 4552, 1063, 4553, 1186, 41],
 [267, 604, 777, 7, 30, 1, 6, 5, 37, 879],
 [4554, 21, 754, 3303, 87, 2009, 1799, 289, 1511, 2681, 3303],
 [27, 202, 510, 76, 190, 5, 37],
 [2, 1512, 2682, 1800, 4555, 927, 4556],
 [1513, 75, 282, 14, 1514, 554],
 [3304, 624, 1337, 132, 3, 33, 1, 155, 5, 37],
 [85],
 [4, 279, 2010, 47, 286, 77, 389, 511, 313, 166],
 [481, 116, 150, 57, 32, 18, 44,

In [11]:
print(sz)

9481


ENCODING LABELS

In [12]:
classes=[]
for lab in labels:
    if lab=='positive':
        classes.append([0,0,1])
    elif lab=='neutral':
        classes.append([0,1,0])
    else :
        classes.append([1,0,0])
classes

[[0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [1, 0, 0],
 [1,

MAX AND MIN LENGTH OF REVIEWS

In [13]:
review_lens = Counter([len(x) for x in coded_revs])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 70
Maximum review length: 43


In [14]:
for i in range(44):
    print(i,review_lens[i])

0 70
1 531
2 659
3 922
4 1019
5 1100
6 1304
7 1344
8 1487
9 1587
10 1517
11 1159
12 880
13 520
14 285
15 133
16 62
17 33
18 14
19 6
20 4
21 0
22 0
23 1
24 0
25 0
26 0
27 1
28 0
29 0
30 1
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 1


REMOVING REVIEWS WITH 0 LENGTH

In [15]:
print('Number of reviews before removing outliers: ', len(coded_revs))

## remove any reviews/labels with zero length from the coded_revs list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(coded_revs) if len(review) != 0]

# remove 0-length reviews and their labels
coded_revs = [coded_revs[ii] for ii in non_zero_idx]
classes = np.array([classes[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(coded_revs))

Number of reviews before removing outliers:  14640
Number of reviews after removing outliers:  14570


FUNTION TO PAD REVIEWS (right)

In [16]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [17]:
seq_length = 15

features = pad_features(coded_revs, seq_length=seq_length)

In [18]:
for i in range(len(coded_revs)):
    print(features[i],classes[i])

[   0    0    0    0    0    0    0    0    0    0   73 3299   18 4547
 4548] [0 1 0]
[   0    0    0  273    4 1334  140  278   13    3  140  193   82    5
   80] [1 0 0]
[   0    0    0    0    0    1  358  553 4549    6  103  132   53  359
  680] [1 0 0]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0 489] [0 0 1]
[  0   0   0   0   0   0   0   0   0   0 326  41 218  52   2] [1 0 0]
[   0    0    0    0    0    0    0  964  220 1335   46  227   51  641
  144] [0 0 1]
[   0    0    0    0    0    0    0    0    0    0 1184  100 3300 3301
  662] [1 0 0]
[   0    0    0    0    0    0    0    0   29  228   11 1336   11  142
    6] [1 0 0]
[   0    0    0    0    0    0    0   24   38 2680  847  642 1798    6
 4550] [1 0 0]
[  0   0   0   0   0   0   0   0   0   0   0   0   0 965 312] [0 0 1]
[   0    0    0    0    0    0    0    0    0  285   49 4551  326   41
 1008] [1 0 0]
[  0   0   0   0   0   0   0   0 248   1 110   5  37  52 602] [0 1 0]
[   0    0    0    0    0    0  

 1675] [0 0 1]
[  0   0   0   0   0   0   0   0   0 470 580  77 100 101 106] [1 0 0]
[   0    0    0    0    0    0   25  130  470  166  652   31  609 1829
   60] [1 0 0]
[   0    0    0    0  685  203  265  102    5   37    1  760 3356 1830
   69] [0 1 0]
[   0    0    0    0    0    0    0    0    0    1   58  426  223  300
 1831] [0 1 0]
[3357 2723   16  120  717  160  560  114  133  278   17 4664 4665    1
  129] [1 0 0]
[   0    0    0  884  572   36 2040 2331   12 2724  572 4666  884 2041
  203] [1 0 0]
[  0   0   0   0 230  45 419  15  81  32  45  89  18  17  53] [1 0 0]
[   0    0    0    0    0    0    0    0    0  652   25 1139 1139 1139
 1139] [0 0 1]
[   0    0    0    0    0    1 1533 1078 2332 2315   75    7  200 2685
 1350] [0 1 0]
[  0   0   0   0   0   0   0   0   0 251  28   7 301  19 826] [1 0 0]
[   0    0    0    0    0    0    0    0    0    0    0    0  216 1351
  983] [0 1 0]
[   0    0   29   22   55   14  330 1676   41    3  293   13   33    1
   79] [1 0 0]
[

[   0    0    0    0    0    0  558   35  179  574    8  384    7  164
 2062] [1 0 0]
[   0    0    0    0    0    0    0    0    0    0    0 1033  309   16
 4755] [0 1 0]
[  0   0   0   0   0   0   0   0   0   0   0  31 520 158 148] [0 0 1]
[   0    0    0    0    0  833   31   11   34  175  122  142  238 1552
 4756] [1 0 0]
[  0   0   0   0   0   0   0   0   0   0   0   0   2 944 373] [0 0 1]
[   0    0    0    0    0  109   89   21 4757 1034 2747   21  352   21
  288] [1 0 0]
[   0    0    0    0    0   16  130   46 3398 2063 2748  625  186  763
   86] [0 1 0]
[  0   0   0   0   0   0   0   0   0   0   0   0  16 298  69] [0 1 0]
[   0    0    0    0    0    0  744 1845   14   99  157 4758  766   79
   95] [1 0 0]
[   0    0    0    0    0    0    0    0   46   83  648  105  632   74
 4759] [0 0 1]
[   0    0    0  212    1 3399 1836  524  241 1035 2749  206  241    1
 3400] [1 0 0]
[  0   0   0   0   0   0   0   0   0 106 705 905  24 870 132] [0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 2] 

[  0   0   0   0   0  16  27   1 717  23  27  92   8   1  93] [1 0 0]
[   0    0    0    0    0 4861 3456  708    1  222  349  203 1699   75
  609] [0 1 0]
[   0    0    0    0    0    0    0    0    0    0    0    7   65 3457
  120] [1 0 0]
[   0    0    0    0    0   73  170  645  580  183  859   77   60  719
 2094] [1 0 0]
[   0    0    0    0    0    0    3    6   55   14 1103  143 1459 1362
   25] [1 0 0]
[  0   0   0   0   0   0   0   0   3 523 567 108   1 276 795] [1 0 0]
[   0    0    0    0    0  180   13    3  822 1380 1460   88  318   35
   18] [1 0 0]
[   0    0    0    0    0    0    0  148  437  852  203   86   35 1706
  411] [1 0 0]
[  0   0   0   0   0   0   0   0   0   0 422 455 455 123 124] [0 1 0]
[   0    0    0    0    0  805 1045  208 1228   26   10   20 1045  121
 1228] [1 0 0]
[  0   0   0   0   0   0   0   0   0   0   0   0   1 300 688] [0 1 0]
[   0    0    0    0    0    0    0    0   18   44    1   15  803   28
 1432] [1 0 0]
[   0    0    0    0  870  975  

[  0   0   0   0   0   0   0   0   0   0   2 453 309 193  42] [1 0 0]
[   0    0  195  289  647  185  165    2  944   17  198   84 1047  244
  465] [0 0 1]
[   0    0    0    0    0    0    0    0    0    2    8   34   25 2837
 2814] [0 0 1]
[   0    0    0    0    0    0    0    0    0    0    0 4922  114  528
  256] [0 1 0]
[   0    0    0    0    0    0    0   22  427 1874 1565  256 1358 4923
   24] [0 1 0]
[   0    0    0    0    0  335  347   84    9    4   57  267   18 3493
   36] [1 0 0]
[   0    0    0    0  466 1160 3494   12   60  153   12  580    1    5
   37] [1 0 0]
[   0    0    0    0    5 1200    1  557   51  606 1680  101  137   97
   28] [1 0 0]
[   0    0    0    0    0    0    0    0   24  459 1348 2405 2122  494
  104] [1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 2] [0 1 0]
[   0    0    0    0    0    0    0    0    0    0   59   36  169 2098
   36] [0 1 0]
[  0   0   0   0   0   0   0   0   0   0   0  16 461  32   1] [1 0 0]
[   0    0    0    0    0    0    0   73   71 

...

[   0    0    0    0    4  160  106 1370 1606  838  549    1   12    5
   80] [1 0 0]
[   0    0    0    0    0    0    0   41  244 2080 1298   25   36  607
 2081] [0 0 1]
[  0   0   0   0   0   0  50 446   9 104 291  50 256 147  84] [1 0 0]
[   0    0    0   22  191    9    4  191 1021  930   79  531 1067  406
  438] [1 0 0]
[   0    0    0    0    0    0    0    0    0    0    0 1510 1073   22
 1793] [1 0 0]
[  0   0   0   2  46   1 412 233 140  46 227 552 244 412 267] [0 0 1]
[  0   0   0   0   0   0   0   0   0   0   0 865   4 122  31] [0 1 0]
[  0   0   0   0   0   1  12   5 192 170 295 378  47  19  19] [1 0 0]
[   0    0    0    0   16 2172   39   35   88  500  235   29  570   38
    1] [1 0 0]
[   0    0    0    2  944   84   30  484  591  880  975 2124 2842  659
   56] [0 0 1]
[  0   0   0   0   0   0  90   8 644 829  54 419 498   8 961] [1 0 0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0 15 14] [0 0 1]
[   0    0    0    0    0    0    0  158  114 1509  132 2051    1  961
  268] [

 9172] [0 1 0]
[  0   0   0   0   0   0   0   0 201 375 129 207  35 386  41] [0 1 0]
[   0    0    0    0    0    0    0    0    0    0  174  156 4541  872
  671] [0 1 0]
[  0   0   0   0   0   0 112 311   6 904  25  43 104 391   9] [1 0 0]
[   0    0    0    0    0   31  740 1126  262 2266    3   55   14 1556
   87] [1 0 0]
[   0    0    0    0    0    0    0    0   30 1475   10  732   41   42
 1032] [1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 2] [0 1 0]
[   0    0    0    0    0    0    0    0   16  157   11  597 1943  965
 1194] [0 1 0]
[   0    0    0  616   12    3   14  140  774  546  225 1967  516  546
 1539] [1 0 0]
[   0    0    0    0    0    0    0   18   40  144  881  691 1239  474
    8] [1 0 0]
[   0    0    0    0    0    0    0    0    0  792 1417 1169 2505 9173
  232] [0 1 0]
[  0   0   0   0   0   0   0   0   0   0   0   0   0  34 424] [0 0 1]
[   0    0    0    0   95 1928   26   10  318  627   26  434    7   26
 2281] [1 0 0]
[  0   0   0   0   0   0   0   0   0 267 160 18

[  0   0   0   0   0  31 164 249  74 396 446  81   9 104 185] [1 0 0]
[   0    0    0    0    0  523 1116    7  787   57 1633  138 9224  751
  120] [1 0 0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 50] [0 1 0]
[   0    0    0    0    0 1003 3396 1335  115  131   79   30   24  131
   40] [1 0 0]
[   0    0    0    0    0    0    0    0    0    0    0 1176  291   18
   44] [1 0 0]
[   0    0    0    0    0    0    0    0 1074  824  195 1258  921 2005
  206] [0 0 1]
[   0    0    0    0    0    0    0    0    0    0    0   73   49 9225
 2157] [0 1 0]
[   0    0    0    0    0  665  162   39   53  726  101 1714  365 9226
   39] [1 0 0]
[   0    0    0    0    0    0    0    0    0    0  287 1623 1408  241
  480] [0 1 0]
[   0    0    0    0    0  384    3  347   15 4130   43  122  145 1319
   92] [1 0 0]
[   0    0    0    0  734    2 9227 1052  658    8  637    3  205 2155
 2480] [0 0 1]
[   0    0    0    0    0  221  144  553  182  134  255   14 9228  255
 1028] [1 0 0]
[   0    0    0

[   0    0    0    0   50  130   67  991   90  404 1001  677 1948 3105
 3972] [0 1 0]
[   0    0    0    0    0    0    0    0    0    0  223   19   36  256
 1079] [0 1 0]
[  0   0   0   0   0   0 404 403 838 772  13  47 102  31 120] [1 0 0]
[   0    0    0    0    0    0    0    0  164  282  989 9301  466   42
   42] [0 1 0]
[  0   0   0   0   0   0   0   0   0   0   0  66 399  66  54] [1 0 0]
[  0   0   0   0   0   2 252  98   1   6 313   6  29 149 589] [0 0 1]
[   0    0    0    0    0    0    0    0    0    0    0 1360    1 9302
  323] [1 0 0]
[   0    0    0    0    0    0    0    0    0  384   19  270   87   14
 9303] [1 0 0]
[  0   0   0   0   0   0   0   0 100  24 106  79 119   5  37] [0 1 0]
[   0    0    0    0    0   13   88    3  269 2283 1598  324   17   35
   17] [1 0 0]
[   0    0    0    0    0    0    0    0    0    0 3213  925  224  123
  124] [1 0 0]
[   0    0    0    0   11    1  256  147 1786   12  288  150 9304 2381
  243] [0 1 0]
[   0    0    0    0    0    0  

[   0    0    0    0    0    0    0    0    0    0    0    0    0 3544
 4434] [0 1 0]
[   0    0    5   80    1 2471  111   17    1   52  897 2471  588    1
 2471] [1 0 0]
[   0    0   32 9469   27    1  200    7   17   42    3   33    1 9470
   48] [1 0 0]
[   0    0    0  195   20 3202 1634 9471   70    1  209 1229  510    5
   37] [0 0 1]
[  0   0   0   0   0   0   0   0   0   0   0   0  91 369  67] [0 1 0]
[   0    0    0    0  840  203  183  278  949 1328    1  336  103 9472
   11] [1 0 0]
[   0    0    0    8 4514   16    8  929 2007 2287 2288 2007 2260 2261
 2007] [0 1 0]
[   0    0    0  215  214 1995   11  421   29   51  106   25   22 1183
  367] [1 0 0]
[   9    4 1150 3364  119    6    5   80   10   54   19 4332  119    6
    5] [1 0 0]
[   0    0    0    0    0  349  988   88   28   13   45  107  235  349
 4546] [1 0 0]
[   0  225 1964 4522  512 1146  169 9473   16 2583 3194   16    8  801
    4] [1 0 0]
[   0    0    0    0  179   81    9 2805  576    9    6    5  495 2266

Saving our features and classes data

In [19]:
from numpy import savetxt
savetxt('features.csv',features,delimiter=',')
savetxt('classes.csv',classes,delimiter=',')